In [1]:
import time
from sympy import *

import numpy as np

In [2]:
x = symbols('x')

# Funciones

In [3]:
def biseccion(a, b, func, tol, iterMax):
    f = lambdify(x,func)
    
    xk=None
    error = abs(f(a+b/2))
    k=iterMax
    
    for i in range(iterMax):
        if f(a)*f(b) < 0:
            xk = (a+b)/2
            error = abs(f(xk))
            if f(xk)*f(a) < 0:
                b=xk
            else:
                a=xk
            if error <= tol:
                k=i
                break
        else:
            xk=None
    
    return xk,error,k

In [4]:
def falsa_posicion(a, b, func, tol, iterMax):
    f = lambdify(x,func)
    
    xk=b
    error = abs(f(a+b/2))
    k=iterMax
    
    x_c = xk
    x_p = a
    
    for i in range(iterMax):
        if f(a)*f(b) < 0:
            xk = xk - (f(x_c)*(x_c - x_p))/(f(x_c)-f(x_p))
            x_p=x_c
            x_c=xk
            error = abs(f(xk))
            if f(xk)*f(a) < 0:
                b=xk
            else:
                a=xk
            if error <= tol:
                k=i
                break
        else:
            xk=None
    
    return xk,error,k

In [5]:
def steffensen_helper(x0, func, iterMax):
    f = lambdify(x, func)
    xk = x0
    
    for i in range(iterMax):
        f_2 = f(xk)**2
        xk = xk - f_2/(f(xk + f(xk))-f(xk))
    
    return xk

In [6]:
def steffensen_secante(x0, func, tol, iterMax):
    f = lambdify(x, func)
    
    xk = x0
    error=f(xk)
    k=iterMax
    
    for i in range(iterMax):
        fx = f(xk)
        funcev1 = fx**3
        funcev2 = f(xk + fx)-fx
        funcev3 = f(xk) - steffensen_helper(x0, func, i)
        xk = xk - funcev1/(funcev2 * funcev3)
        error = abs(f(xk))
        if error < tol:
            k=i
            break
    
    return xk, error, k

In [7]:
def noor_one_point(f, x0, tol, iterMax):
    """

    Función de Noor's One-Point para encontrar la raíz de una ecuación.
    
    Parámetros:
        f: Función a la que se le buscará la raíz.
        x0: Aproximación inicial.
        tol: Tolerancia para detener el método.
        iterMax: Número máximo de iteraciones permitidas.
    
    Resultados:
        x: Aproximación de la raíz encontrada.
        error: Valor absoluto de f(x), que se toma como el error en la aproximación.
        k: Número de iteraciones realizadas.
        tiempo: Tiempo de ejecución del método en segundos.
    """
    
    # Iniciar el temporizador para medir el tiempo de ejecución
    start_time = time.time()
    
    # Inicializar variables
    x = x0
    k = 0           # Contador de iteraciones
    error = float('inf')     # Error inicial (inicializado en infinito)
    
    # Iterar hasta alcanzar el número máximo de iteraciones
    while k < iterMax:
        # Calcular el valor de la función en la aproximación actual
        f_x = f(x)
        
        # Calcular la derivada numérica en la aproximación actual utilizando diferencias finitas
        f_prime_x = (f(x + tol) - f(x - tol)) / (2 * tol)

        # Manejar el caso donde la derivada es aproximadamente cero
        if abs(f_prime_x) < 1e-10:
            # Evitar la división por cero, ajustando ligeramente la derivada
            f_prime_x = 1e-10
        
        # Actualizar la aproximación utilizando el método de Noor's One-Point de segundo orden
        x = x - f_x / f_prime_x
        
        # Calcular el nuevo error absoluto (valor absoluto de f(x))
        error = abs(f_x)
        
        # Comprobar si se ha alcanzado la tolerancia deseada
        if error < tol:
            break  # Salir del bucle si se cumple la condición de parada
        
        # Incrementar el contador de iteraciones
        k += 1
    
    # Detener el temporizador y registrar el tiempo de ejecución
    tiempo = time.time() - start_time
    
    return x, error, k, tiempo
    

In [8]:
def f(x):
    return exp(x)-x-10

# Testing

In [9]:
start = time.time()
xk, err, k = biseccion(-2,3, exp(x)-x-10,1e-10, 1000)
stop = time.time()
print(f"Tiempo de ejecución: {stop - start}s")

print("Xk = " + str(xk))
print("err = " + str(err))
print("k = " + str(k))

Tiempo de ejecución: 1.1940767765045166s
Xk = 2.527963201981038
err = 1.3100631690576847e-11
k = 30


In [10]:
start = time.time()
xk, err, k = falsa_posicion(-2,3, exp(x)-x-10,1e-10, 1000)
stop = time.time()
print(f"Tiempo de ejecución: {stop - start}s")

print("Xk = " + str(xk))
print("err = " + str(err))
print("k = " + str(k))

Tiempo de ejecución: 0.003509998321533203s
Xk = 2.5279632019811604
err = 1.1686651646414248e-11
k = 9


In [11]:
start = time.time()
xk, err, k = steffensen_secante(3, exp(x)-x-10,1e-10, 1000)
stop = time.time()
print(f"Training time: {stop - start}s")

print("Xk = " + str(xk))
print("err = " + str(err))
print("k = " + str(k))

C:\Users\yarit\AppData\Local\Temp\ipykernel_6516\3948318365.py:7: RuntimeWarning: invalid value encountered in double_scalars
  xk = xk - f_2/(f(xk + f(xk))-f(xk))


Training time: 5.948085308074951s
Xk = nan
err = nan
k = 1000


In [12]:
# Llama a la función steffensen_secant
x0 = 1.5
tol = 1e-10
iterMax = 1000

start = time.time()
xk, err, k, tiempo_ejec = noor_one_point(f, x0, tol, iterMax)
stop = time.time()

print("Xk:", xk)
print("Error:", err)
print("Número de iteraciones:", k)
print("Tiempo de ejecución:", "{:.10f}".format(tiempo_ejec), "segundos")
print(f"Training time: {stop - start}s")

Xk: 2.52796320198217
Error: 4.38120650869678e-11
Número de iteraciones: 6
Tiempo de ejecución: 0.0016348362 segundos
Training time: 0.001634836196899414s
